# Taxi Trip Dataset — San Francisco  
*(Data preparation notebook – section 1)*  

## 1. Overview  

This notebook starts the data‑wrangling pipeline for San Francisco taxi trips and serves as the **mirror dataset** for the Waymo rides we analysed earlier.  
The raw file was downloaded directly from the official **SFMTA open‑data portal** and covers the **most recent complete window that overlaps with our Waymo sample**:

| Service | Coverage window | Note |
|---------|-----------------|------|
|Waymo One (AV) | **15 Jan 2024 – 10 Mar 2024** | Public ride‑hailing pilot |
|SFMTA Taxi Feed | **15 Jan 2023 – 10 Mar 2023** | Latest full release available (one‑year‑back mirror) |

> **Why use the previous year?**  
> The 2024 taxi feed has not yet been published by SFMTA. Using the same calendar window from 2023 ensures seasonal comparability (weather, events, demand) while giving us the freshest officially released data.

### 1.1 Services included  

* **Flywheel Taxi** – e‑hail and dispatch trips  
* **UberTaxi / Uber TaxiOption** – medallion cabs hailed via the Uber app  
* **Street‑hail (“flag‑drop”)** – traditional curbside pickups  

All records are anonymised but contain:

* precise pickup / drop‑off coordinates  
* trip start / end timestamps (local PST)  
* meter‑based fare components (distance, time, surcharges)  
* additional fields: tolls, airport fee, tips, promo discounts, upfront‑pricing flag

### 1.2 Goals of this section  

1. **Load** the raw CSV / Parquet into a working DataFrame.  
2. **Verify** schema, dtypes and basic ranges (latitude, longitude, fare, speed).  
3. **Document** the source and time window so the notebook is self‑contained before we proceed to cleaning, feature engineering and alignment with Waymo data.

> *Next steps:*  
> In upcoming sections we’ll clean outliers, align fare calculations, and spatially filter trips to Waymo’s current service area.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os
import math
import sys
sys.path.append('/Users/pavlova92/Desktop/Springboard') 
from sb_utils import save_file
from datasets import load_dataset
from ydata_profiling import ProfileReport


In [2]:
sf = pd.read_csv('/Users/pavlova92/Downloads/Taxi_Trips_20250720-2.csv')
sf.head()#/Users/pavlova92/Downloads/Taxi_Trips_20250720-2.csv

/var/folders/s0/mzj0z0s5637cx3xl_5yngt_00000gn/T/ipykernel_77437/1793923671.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  sf = pd.read_csv('/Users/pavlova92/Downloads/Taxi_Trips_20250720-2.csv')


,vehicle_placard_number,driver_id,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,pickup_location,dropoff_location_latitude,dropoff_location_longitude,dropoff_location,...,tolls,sf_exit_fee,other_fees,tip,extra_amount,total_fare_amount,fare_time_milliseconds,trip_distance_meters,data_as_of,data_loaded_at
0,1376,-,2024/02/05 02:13:50 AM,2024/02/05 02:23:48 AM,37.764,-122.419,POINT (-122.419 37.764),37.712,-122.453,POINT (-122.453 37.712),...,NaN,NaN,NaN,0.0,NaN,19.75,597723,7402.9824,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
1,1376,D-1632,2024/02/05 02:13:50 AM,2024/02/05 02:23:48 AM,37.764,-122.419,POINT (-122.419 37.764),37.712,-122.453,POINT (-122.453 37.712),...,NaN,NaN,NaN,0.0,NaN,19.75,597723,7403.0000,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
2,49,D-0091,2024/02/05 02:14:36 AM,2024/02/05 02:38:13 AM,37.617,-122.386,POINT (-122.386 37.617),37.456,-122.132,POINT (-122.132 37.456),...,NaN,NaN,0.0,0.0,10.0,82.40,1114000,32566.1600,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
3,148,B-8324,2024/02/05 02:14:38 AM,2024/02/05 02:29:38 AM,37.614,-122.385,POINT (-122.385 37.614),37.772,-122.419,POINT (-122.419 37.772),...,NaN,NaN,0.0,0.0,5.5,53.20,790000,21287.0680,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
4,1401,A-2359,2024/02/05 02:14:57 AM,2024/02/05 03:28:11 AM,37.617,-122.386,POINT (-122.386 37.617),37.615,-122.390,POINT (-122.39 37.615),...,NaN,NaN,0.0,0.0,5.5,118.85,2480000,46435.7420,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM


In [3]:
report_raw = ProfileReport(sf, title="San Francisco Raw Data Report")
report_raw.to_file("San_Francisco_Raw_Data_Report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 28/28 [00:42<00:00,  1.51s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# Load the Waymo estimates dataset
waymo = load_dataset("purav/waymo_ride_estimates")  
waymo = waymo['train'].to_pandas()
import geopandas as gpd, pandas as pd, folium
from shapely.geometry import Point, LineString
# download the neighborhoods GeoJSON file
hoods = gpd.read_file('/Users/pavlova92/Downloads/archive-14/sf_neighborhoods.geojson').to_crs('EPSG:4326')
# GeoDataFrame for pickup locations
waymo_gdf = gpd.GeoDataFrame(
    waymo,
    geometry=gpd.points_from_xy(waymo.pickup_longitude, waymo.pickup_latitude),
    crs='EPSG:4326'
)

# second GeoDataFrame for drop‑off locations
waymo_drop = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(waymo.dropoff_longitude, waymo.dropoff_latitude),
    crs='EPSG:4326'
)
# neihborhoods where Waymo serves
waymo_with_hood = waymo_gdf.sjoin(
    hoods[['name', 'geometry']],
    predicate='within', how='left'
)

served_hoods = waymo_with_hood['name'].dropna().unique().tolist()

# filter Waymo rides to only those that start and end in neighborhoods served by Waymo
waymo_area = hoods.query('name in @served_hoods')
pick_ok = waymo_gdf.sjoin(waymo_area[['geometry']], predicate='within', how='inner').index
drop_ok = waymo_drop.sjoin(waymo_area[['geometry']], predicate='within', how='inner').index
df_waymo = waymo.loc[pick_ok.intersection(drop_ok)]
# SF map (with tghe center in the middle of the city)
center = [37.77, -122.42]
m = folium.Map(location=center, zoom_start=16, tiles='cartodbpositron')

# waymo service coverage area
folium.GeoJson(
    waymo_area,
    name='Waymo service area',
    style_function=lambda x: {"fillColor": "#74c476", "color": "#238b45", "weight":1, "fillOpacity":0.2}
).add_to(m)
# JOIN for the neighborhoods

# waymo-pickup points for  2000 rides 
for _, r in df_waymo.head(2000).iterrows():         
    folium.CircleMarker(
        location=[r.pickup_latitude, r.pickup_longitude],
        radius=2, color='#08519c', fill=True, fill_opacity=0.7
    ).add_to(m)
from shapely.geometry import LineString

def as_line(row):
    return LineString([(row.pickup_lonitude, row.pickup_latitude),
                       (row.dropoff_longitude, row.dropoff_latitude)])
folium.LayerControl().add_to(m)
m.save('waymo_map.html')

In [5]:
#defining a function for basic data exploriation with the main methods of EDA 
def data_exploiation(dataset):
    print(" Shape:")
    print(f"Rows: {dataset.shape[0]} | Columns: {dataset.shape[1]}")
    print("\n Column Names:")
    print(dataset.columns)
    
    print("\n Data Types:")
    print(dataset.dtypes)
    
    print("\n Missing Values:")
    missing = dataset.isnull().sum()
    print(missing[missing > 0].sort_values(ascending=False))
    
    print("\n Memory Usage:")
    mem = dataset.memory_usage(deep=True).sum() / (1024 ** 2)
    print(f"{mem:.2f} MB")
    
    print("\n Descriptive Stats (Numerical):")
    print(dataset.describe().T)
    
    print("\n Unique Values per Column:")
    print(dataset.nunique())
    
    
    print("\n Duplicate Rows:")
    print(dataset.duplicated().sum())

In [6]:
# dataframe with taxi trips in San Francisco
taxi_pick = gpd.GeoDataFrame(
    sf,
    geometry=gpd.points_from_xy(sf.pickup_location_longitude, sf.pickup_location_latitude),
    crs='EPSG:4326'
)
# GeoDataFrame for SF taxi pickup and drop‑off locations
taxi_in = taxi_pick.sjoin(waymo_area[['geometry']], predicate='within', how='inner')
pick_ok_idx = taxi_in.index
taxi_drop = gpd.GeoDataFrame(geometry=gpd.points_from_xy(sf.dropoff_location_longitude, sf.dropoff_location_latitude),crs='EPSG:4326')
taxi_out  = taxi_drop.sjoin(waymo_area[['geometry']], predicate='within', how='inner')
drop_ok_idx = taxi_out.index

# final taxi dataset with trips that start and end in neighborhoods served by Waymo
taxi_in_waymo_zone = sf.loc[pick_ok_idx.intersection(drop_ok_idx)]

taxi_in_waymo_zone.to_csv('taxi_in_waymo_zone.csv', index=False)

In [7]:
df_sf =pd.read_csv('taxi_in_waymo_zone.csv')
df_sf.head()

/var/folders/s0/mzj0z0s5637cx3xl_5yngt_00000gn/T/ipykernel_77437/2486925358.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sf =pd.read_csv('taxi_in_waymo_zone.csv')


,vehicle_placard_number,driver_id,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,pickup_location,dropoff_location_latitude,dropoff_location_longitude,dropoff_location,...,tolls,sf_exit_fee,other_fees,tip,extra_amount,total_fare_amount,fare_time_milliseconds,trip_distance_meters,data_as_of,data_loaded_at
0,1376,-,2024/02/05 02:13:50 AM,2024/02/05 02:23:48 AM,37.764,-122.419,POINT (-122.419 37.764),37.712,-122.453,POINT (-122.453 37.712),...,NaN,NaN,NaN,0.0,NaN,19.75,597723,7402.9824,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
1,1376,D-1632,2024/02/05 02:13:50 AM,2024/02/05 02:23:48 AM,37.764,-122.419,POINT (-122.419 37.764),37.712,-122.453,POINT (-122.453 37.712),...,NaN,NaN,NaN,0.0,NaN,19.75,597723,7403.0000,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
2,30,-,2024/02/05 02:16:08 AM,2024/02/05 02:20:15 AM,37.772,-122.429,POINT (-122.429 37.772),37.767,-122.450,POINT (-122.45 37.767),...,NaN,NaN,NaN,0.0,NaN,8.05,246252,1931.2128,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
3,30,N-4473,2024/02/05 02:16:08 AM,2024/02/05 02:20:15 AM,37.772,-122.429,POINT (-122.429 37.772),37.767,-122.450,POINT (-122.45 37.767),...,NaN,NaN,NaN,0.0,NaN,8.05,246252,1931.2000,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM
4,686,A-4984,2024/02/05 02:17:28 AM,2024/02/05 02:31:50 AM,37.785,-122.404,POINT (-122.404 37.785),37.788,-122.450,POINT (-122.45 37.788),...,0.0,0.0,0.0,0.0,0.0,19.94,0,5590.2450,2025/04/04 10:37:04 AM,2025/04/07 04:16:16 PM


In [8]:
#defining a function for basic data exploriation with the main methods of EDA 
def data_exploiation(dataset):
    print(" Shape:")
    print(f"Rows: {dataset.shape[0]} | Columns: {dataset.shape[1]}")
    print("\n Column Names:")
    print(dataset.columns)
    
    print("\n Data Types:")
    print(dataset.dtypes)
    
    print("\n Missing Values:")
    missing = dataset.isnull().sum()
    print(missing[missing > 0].sort_values(ascending=False))
    
    print("\n Memory Usage:")
    mem = dataset.memory_usage(deep=True).sum() / (1024 ** 2)
    print(f"{mem:.2f} MB")
    
    print("\n Descriptive Stats (Numerical):")
    print(dataset.describe().T)
    
    #print("\n Unique Values per Column:")
    #print(dataset.nunique())
    
    
    print("\n Duplicate Rows:")
    print(dataset.duplicated().sum())

In [9]:
data_exploiation(df_sf)

 Shape:
Rows: 353010 | Columns: 28

 Column Names:
Index(['vehicle_placard_number', 'driver_id', 'start_time_local',
       'end_time_local', 'pickup_location_latitude',
       'pickup_location_longitude', 'pickup_location',
       'dropoff_location_latitude', 'dropoff_location_longitude',
       'dropoff_location', 'hail_type', 'paratransit', 'sfo_pickup',
       'qa_flags', 'fare_type', 'meter_fare_amount', 'upfront_pricing',
       'promo_rate', 'tolls', 'sf_exit_fee', 'other_fees', 'tip',
       'extra_amount', 'total_fare_amount', 'fare_time_milliseconds',
       'trip_distance_meters', 'data_as_of', 'data_loaded_at'],
      dtype='object')

 Data Types:
vehicle_placard_number         object
driver_id                      object
start_time_local               object
end_time_local                 object
pickup_location_latitude      float64
pickup_location_longitude     float64
pickup_location                object
dropoff_location_latitude     float64
dropoff_location_longitude  

In [10]:
# Converting columns to appropriate data types:
# 1. numeric columns
num_cols = ['vehicle_placard_number', 
    'pickup_location_latitude', 'pickup_location_longitude',
    'dropoff_location_latitude', 'dropoff_location_longitude',
    'meter_fare_amount', 'other_fees', 'tip', 'extra_amount',
    'total_fare_amount', 'fare_time_milliseconds',
    'trip_distance_meters', 'tolls', 'sf_exit_fee',
    'upfront_pricing', 'promo_rate']
for col in num_cols:
    df_sf[col] = pd.to_numeric(df_sf[col], errors='coerce')
# datetime columns
time_cols = ['start_time_local', 'end_time_local']
for col in time_cols:
    df_sf[col]  = pd.to_datetime(df_sf[col], format="%Y/%m/%d %I:%M:%S %p")
# categorical columns
cat_cols = ['hail_type', 'paratransit', 'fare_type']
for col in cat_cols:
    df_sf[col] = df_sf[col].astype('category')

In [11]:
# checking if the data types have been changed correctly
df_sf.dtypes

vehicle_placard_number               float64
driver_id                             object
start_time_local              datetime64[ns]
end_time_local                datetime64[ns]
pickup_location_latitude             float64
pickup_location_longitude            float64
pickup_location                       object
dropoff_location_latitude            float64
dropoff_location_longitude           float64
dropoff_location                      object
hail_type                           category
paratransit                         category
sfo_pickup                             int64
qa_flags                              object
fare_type                           category
meter_fare_amount                    float64
upfront_pricing                      float64
promo_rate                           float64
tolls                                float64
sf_exit_fee                          float64
other_fees                           float64
tip                                  float64
extra_amou

In [12]:
df_sf.drop(columns = ['vehicle_placard_number', 'driver_id', 'pickup_location', 'dropoff_location', 'paratransit', 'fare_type', 'data_as_of', 'data_loaded_at', 'qa_flags'], inplace=True)
# dropping columns that are not needed for analysis

In [13]:
df_sf[df_sf['pickup_location_latitude'].isna() | df_sf['pickup_location_longitude'].isna()].shape
# checking for trips with missing pickup or dropoff coordinates

(0, 19)

In [14]:
df_sf.dropna(subset=['pickup_location_latitude', 'pickup_location_longitude', 'dropoff_location_latitude' ,'dropoff_location_longitude', 'hail_type'], inplace=True)
# dropping trips with missing pickup or dropoff coordinates (if any) and no hail_type data

In [15]:
df_sf.isna().sum()

start_time_local                   0
end_time_local                     0
pickup_location_latitude           0
pickup_location_longitude          0
dropoff_location_latitude          0
dropoff_location_longitude         0
hail_type                          0
sfo_pickup                         0
meter_fare_amount                  0
upfront_pricing               293916
promo_rate                    327610
tolls                         287292
sf_exit_fee                   327610
other_fees                    223743
tip                                0
extra_amount                  235832
total_fare_amount                  0
fare_time_milliseconds             0
trip_distance_meters               0
dtype: int64

In [16]:
df_null= df_sf[df_sf['promo_rate'].isna()]
df_null.isna().sum()
# checking if nan values in pricing colimns are related  by picking the column with the most nan values and confirming that the rest of the columns are also nan

start_time_local                   0
end_time_local                     0
pickup_location_latitude           0
pickup_location_longitude          0
dropoff_location_latitude          0
dropoff_location_longitude         0
hail_type                          0
sfo_pickup                         0
meter_fare_amount                  0
upfront_pricing               293916
promo_rate                    327610
tolls                         287292
sf_exit_fee                   327610
other_fees                    223743
tip                                0
extra_amount                  235832
total_fare_amount                  0
fare_time_milliseconds             0
trip_distance_meters               0
dtype: int64

In [17]:
aligned = ((df_null['sfo_pickup'] == 0) == df_null['sf_exit_fee'].isna()).all()
if aligned:
    print("Every pickup==0 row has sf_exit_fee = NaN, and vice versa")
else:
    print("Mismatch found; investigate specific rows")
# we confirming that the number of rows with no SFO pickup is  to the number of rows with no SF exit fee, so we can drop those rows since Waymo service area excludes airport location

Every pickup==0 row has sf_exit_fee = NaN, and vice versa


In [18]:
df_null['fare_diff'] = df_null['total_fare_amount'] - df_null['meter_fare_amount'] 
df_null['fare_diff'].describe()

#  checking the fare difference between total fare and meter fare for trips with missing price componenets to see nan values are related to fare difference or not:
#  most of them are 0, so we can assume that nan = 0

/var/folders/s0/mzj0z0s5637cx3xl_5yngt_00000gn/T/ipykernel_77437/3538251026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_null['fare_diff'] = df_null['total_fare_amount'] - df_null['meter_fare_amount']


count    327610.000000
mean          0.886828
std           3.078819
min         -20.550000
25%           0.000000
50%           0.000000
75%           0.000000
max         330.000000
Name: fare_diff, dtype: float64

In [19]:
df_sf.fillna({'upfront_pricing': 0, 'promo_rate':0, 'tolls': 0, 'other_fees': 0, 'tip': 0, 'extra_amount': 0}, inplace=True)
# filling missing values in fare components with 0, as we assume that they are not charged for these trips
df_sf.drop(columns = ['sf_exit_fee'], inplace=True)
# dropping sfo_exit_fee column as it duplicates sf_exit_fee column ($5.50 it is charged only for trips with sfo_pickup == 1)

In [20]:
df_sf['summary_fees'] =df_sf['tolls']+df_sf['other_fees']+df_sf['tip'] + df_sf['extra_amount']+df_sf['promo_rate'] 
# calculating the the sum of all fare components 

In [21]:
df_sf['trip_distance_miles'] = df_sf['trip_distance_meters'] / 1609.34
# calculating trip_distance in miles
df_sf.drop(columns=['trip_distance_meters'], inplace =True)
# dropping trip_distance_meters column as it is no longer neede

In [22]:
df_sf['trip_duration_min'] = (df_sf['end_time_local'] - df_sf['start_time_local']).dt.total_seconds()/60
df_sf= df_sf[df_sf['trip_duration_min'] >=0].copy()
# calculating trip_duration in minutes and removing trips with negative duration

In [23]:
df_sf = df_sf[df_sf['fare_time_milliseconds']>=0].copy()
# removing trips with negative fare_time_milliseconds

In [24]:
df_sf['delay'] = df_sf['trip_duration_min']*60- df_sf['fare_time_milliseconds'] /1000 
df_sf['delay'].describe().T
# calculating trip time delay in minutes and checking its distribution

count    353007.000000
mean        188.312313
std         364.353860
min       -1830.000000
25%          -0.104000
50%           0.346000
75%         306.000000
max       46542.000000
Name: delay, dtype: float64

In [25]:
df_sf[df_sf['delay']<-1].shape[0]/ df_sf.shape[0] * 100
# checking the percentage of trips where wait-traffic  time in seconds is negative and its absolute value exceeds 1 second 

0.0002832805015197999

In [26]:
df_sf = df_sf[df_sf['delay']>=-1].copy()
# removing trips with negative delay and its absolute value is less than 1 second

In [27]:
df_sf['delay_min'] = np.where(df_sf['delay']<0, 0, df_sf['delay']/60)
# replacing negative delays with 0, as they are a result of data errors (most of those values are less than 1 second) and transforming the delay to minutes
df_sf.drop(columns=['delay'], inplace=True)
# dropping delay column as it is no longer needed
df_sf['delay_min'].describe().T

count    353006.000000
mean          3.140363
std           6.071414
min           0.000000
25%           0.000000
50%           0.005767
75%           5.100000
max         775.700000
Name: delay_min, dtype: float64

In [28]:
# df_sf already has:
#  trip_distance_miles, delay_min, sfo_pickup
#  tolls, other_fees, extra_amount, promo_rate, tip
#  upfront_pricing, total_fare_amount
#  the official SFMTA San Francisco Taxicab Rates of Fare formula: First 1/5th mile or flag rate: 
# $4.15*
#Each additional 1/5th mile or fraction thereof……….…… $0.65*
#Each minute of waiting or traffic time delay……………… $0.65*
#Pick-up fee at San Francisco International Airport  5.5 ..…… $6.00** from 1st of July 2025 (if applicable)
step = 0.2
flag_rate    = 4.15
dist_tick    = 0.65
time_tick    = 0.65
sfo_fee_rate  = 5.5               # fee in force for this dataset ($6.00** from 1st of July 2025 (if applicable)

df_sf['price_calculated'] = (flag_rate + np.ceil(np.maximum(df_sf['trip_distance_miles'] - step, 0) / step) * dist_tick 
             + df_sf['delay_min'] * time_tick
             + df_sf['sfo_pickup'] * sfo_fee_rate
)


In [29]:
# calculating the summary price based on the calculated price and summary fees 
df_sf['price_calculated_total'] = df_sf['price_calculated'] + df_sf['summary_fees'] + df_sf['tip']
df_sf.drop(columns=['price_calculated'], inplace=True)
# dropping price_calculated column as it is no longer needed
df_sf['price_calculated_total'].describe().T

count    353006.000000
mean         13.885277
std           9.453622
min           4.150000
25%           7.550000
50%          11.302638
75%          17.204167
max         512.905000
Name: price_calculated_total, dtype: float64

In [30]:
df_sf.describe().T

,count,mean,min,25%,50%,75%,max,std
start_time_local,353006,2024-02-13 05:44:28.574412800,2024-01-15 03:33:12,2024-02-01 12:17:48.500000,2024-02-13 17:55:41.500000,2024-02-25 17:23:35.500000,2024-03-07 15:28:21,NaN
end_time_local,353006,2024-02-13 05:56:23.397678848,2024-01-15 03:39:49,2024-02-01 12:29:39,2024-02-13 18:07:01,2024-02-25 17:33:49,2024-03-07 15:30:24,NaN
pickup_location_latitude,353006.0,37.776544,37.709,37.766,37.782,37.788,37.81,0.019007
pickup_location_longitude,353006.0,-122.421202,-122.513,-122.431,-122.417,-122.407,-122.367,0.021569
dropoff_location_latitude,353006.0,37.775465,37.709,37.765,37.781,37.789,37.81,0.020088
dropoff_location_longitude,353006.0,-122.422455,-122.513,-122.432,-122.418,-122.407,-122.364,0.022803
sfo_pickup,353006.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
meter_fare_amount,353006.0,13.445996,0.0,8.7,11.95,16.5,492.95,9.247249
upfront_pricing,353006.0,2.555638,0.0,0.0,0.0,0.0,89.97,6.231002
promo_rate,353006.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Quick Stats & Data‑Quality Flags  

| Column                | min | 25 % | 50 % | 75 % | **max** | Comment                          |
|-----------------------|----:|-----:|-----:|-----:|--------:|----------------------------------|
|`trip_duration_min`    | 0   | 7.4 | 9.8 | 14.6 | **2714**| 45 h → impossible for a city cab |
|`moving_time_min`      | 0   | 6.1 | 8.2 | 12.1 | **2714**| meter kept “moving” two days     |
|`delay_min`            | 0   | 0.8 | 1.9 | 4.7  | **776** | 13 h idle with near‑zero distance|

> **Observation:** Each time‑related field contains extreme values that do not reflect plausible taxi operation and will skew averages or model targets.

### Next steps

1. **Inspect each column individually**  
   * `trip_duration_min` — identify rides beyond the 99‑th percentile.  
   * `moving_time_min` — detect “stuck‑meter” records.  
   * `delay_min` — flag unusually large or suspiciously small delays related to extremely short rides with unusually high price.

2. **Feature engineering:** 
   * actual moving time,
   * speed (mph) based on actual moving time
3.  **Build filtering rules** based on  
   * the physical minimum speed (12 mph),  
   * quantile thresholds (P99) for duration and delay 


In [31]:
df_sf['moving_time_min'] = df_sf['fare_time_milliseconds'] / 60000
df_sf.drop(columns = 'fare_time_milliseconds', inplace=True)
df_sf['moving_time_min'].describe().T
# calculating moving time in minutes based on fare_time_milliseconds and checking its distribution

count    353006.000000
mean          8.775087
std          18.721845
min           0.000000
25%           2.369717
50%           6.850000
75%          12.242521
max        2713.111267
Name: moving_time_min, dtype: float64

In [32]:
df_sf['speed_mph'] = np.where(df_sf['moving_time_min']>0 , df_sf['trip_distance_miles'] / (df_sf['moving_time_min'] / 60), 0)
# calculating speed in miles per hour
df_sf['speed_mph'].describe().T

count    353006.000000
mean         11.874485
std          85.105581
min           0.000000
25%           5.440184
50%           9.615788
75%          15.340590
max       22226.270370
Name: speed_mph, dtype: float64

In [33]:
df_sf.sort_values(by='speed_mph', ascending=False).head(20)

,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,dropoff_location_latitude,dropoff_location_longitude,hail_type,sfo_pickup,meter_fare_amount,upfront_pricing,...,tip,extra_amount,total_fare_amount,summary_fees,trip_distance_miles,trip_duration_min,delay_min,price_calculated_total,moving_time_min,speed_mph
231583,2024-03-03 21:35:19,2024-03-03 21:35:40,37.785,-122.410,37.785,-122.410,street,0,51.60,0.0,...,0.0,0.0,57.10,5.5,13.700026,0.350000,0.313017,54.053461,0.036983,22226.270370
205373,2024-03-02 00:16:22,2024-03-02 00:16:37,37.801,-122.443,37.801,-122.443,street,0,60.05,0.0,...,0.0,0.0,65.55,5.5,16.299850,0.250000,0.197750,62.428537,0.052250,18717.530673
50695,2024-01-18 09:04:04,2024-01-18 09:04:12,37.762,-122.412,37.762,-122.412,street,0,46.40,0.0,...,0.0,0.0,51.90,5.5,12.400114,0.133333,0.088450,50.007493,0.044883,16576.461789
352741,2024-02-29 22:39:19,2024-02-29 22:39:24,37.772,-122.509,37.772,-122.509,street,0,58.75,0.0,...,6.0,0.0,70.25,11.5,16.199809,0.083333,0.016667,73.660833,0.066667,14579.827755
314512,2024-02-25 17:37:47,2024-02-25 17:37:51,37.789,-122.406,37.789,-122.406,street,0,50.95,0.0,...,0.0,5.5,56.45,5.5,13.900046,0.066667,0.003667,54.502383,0.063000,13238.139030
314513,2024-02-25 17:37:47,2024-02-25 17:37:51,37.789,-122.406,37.789,-122.406,street,0,50.95,0.0,...,0.0,5.5,56.45,5.5,13.900035,0.066667,0.003667,54.502383,0.063000,13238.128141
62314,2024-02-11 19:11:25,2024-02-11 19:11:29,37.782,-122.404,37.782,-122.404,street,0,50.95,0.0,...,0.0,5.5,56.45,5.5,14.100066,0.066667,0.000000,55.150000,0.071600,11815.697653
62316,2024-02-11 19:11:25,2024-02-11 19:11:29,37.782,-122.404,37.782,-122.404,street,0,50.95,0.0,...,0.0,5.5,56.45,5.5,14.100035,0.066667,0.000000,55.150000,0.071600,11815.671826
324344,2024-02-26 14:24:39,2024-02-26 14:25:42,37.797,-122.400,37.797,-122.400,street,0,56.15,0.0,...,0.0,0.0,61.65,5.5,14.499733,1.050000,0.964150,57.076698,0.085850,10133.767834
129044,2024-01-25 22:19:09,2024-01-25 22:19:19,37.768,-122.451,37.768,-122.451,street,0,19.10,0.0,...,0.0,0.0,19.10,0.0,3.600234,0.166667,0.139267,15.940523,0.027400,7883.723291


In [34]:
threshold_speed = df_sf['speed_mph'].quantile(0.99).round(2)
threshold_speed
# setting the threshold for speed based on the 99th percentile of speed distribution

33.7

In [35]:
df_sf= df_sf[df_sf['speed_mph'] <= threshold_speed].copy()
# removing trips with speed above the threshold

### Insight: Distance ≫ Duration

* Numerous records show **large mileage but only a few minutes of travel time**.  
* Such speed is physically impossible → most likely GPS spikes or timestamp errors.  
* **Action:** mark as anomalies and drop from the clean dataset.

In [36]:
df_sf.sort_values(by='trip_duration_min', ascending=False).head(20)

,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,dropoff_location_latitude,dropoff_location_longitude,hail_type,sfo_pickup,meter_fare_amount,upfront_pricing,...,tip,extra_amount,total_fare_amount,summary_fees,trip_distance_miles,trip_duration_min,delay_min,price_calculated_total,moving_time_min,speed_mph
119863,2024-02-14 20:37:54,2024-02-16 17:51:01,37.785,-122.407,37.783,-122.419,street,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2713.116667,0.005400,4.153510,2713.111267,0.000000
119862,2024-02-14 20:37:54,2024-02-16 17:51:01,37.785,-122.407,37.783,-122.419,street,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2713.116667,0.005400,4.153510,2713.111267,0.000000
119864,2024-02-14 20:37:54,2024-02-16 17:51:01,37.785,-122.407,37.783,-122.419,street,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2713.116667,0.005400,4.153510,2713.111267,0.000000
45571,2024-02-10 13:25:24,2024-02-12 08:26:05,37.807,-122.407,37.805,-122.421,street,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2580.683333,0.008483,4.155514,2580.674850,0.000000
45570,2024-02-10 13:25:24,2024-02-12 08:26:05,37.807,-122.407,37.805,-122.421,street,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2580.683333,0.008483,4.155514,2580.674850,0.000000
71942,2024-01-20 17:54:04,2024-01-22 11:10:53,37.782,-122.412,37.768,-122.402,dispatch,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2476.816667,0.000000,4.150000,2476.822617,0.000000
80660,2024-01-22 07:37:39,2024-01-23 19:30:41,37.781,-122.503,37.786,-122.453,dispatch,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2153.033333,0.000000,4.150000,2153.037267,0.000000
80658,2024-01-22 07:37:39,2024-01-23 19:30:41,37.781,-122.503,37.786,-122.453,dispatch,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,2153.033333,0.000000,4.150000,2153.037267,0.000000
12140,2024-02-07 11:08:29,2024-02-08 10:52:42,37.746,-122.403,37.739,-122.406,street,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,1424.216667,0.004017,4.152611,1424.212650,0.000000
13041,2024-02-07 13:08:26,2024-02-08 07:52:27,37.763,-122.457,37.725,-122.427,dispatch,0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.000000,1124.016667,0.000000,4.150000,1124.018817,0.000000


### Insight: “Long‑Duration, Zero‑Delay” Anomaly  

> Sorting the data by `trip_duration_min` in descending order shows that **most of the extremely long trips have `delay_min ~ 0` as well as `trip_distance_miles ~ 0`**.  
> That pattern strongly suggests a **stuck meter**: the cab was stationary (or moving very slowly) for hours, but the taximeter never switched to the waiting‑time mode.

**Implication:**  
These records are not genuine long journeys; they artificially inflate total fare and travel‑time statistics. We will drop them in the outlier‑cleaning step.

In [37]:
threshold_delay= df_sf['delay_min'].quantile(0.99).round(2)
threshold_delay
# setting the threshold for delay based on the 99th percentile of delay distribution

23.15

In [38]:
threshold_distance = df_sf['trip_distance_miles'].quantile(0.99).round(2)
threshold_distance
# setting the threshold for distance based on the 99th percentile of distance distribution

7.93

In [39]:

threshold_duration = df_sf['trip_duration_min'].quantile(0.99).round(2)
threshold_duration
# setting the threshold for trip duration based on the 99th percentile of trip duration distribution

39.55

In [40]:
min_distance_city = (12*threshold_duration/60).round(2)
# setting the minimum distance for city trips based on 12 mph min moving speed ( according to the SFMTA San Francisco https://data.sfgov.org/Transportation/Taxi-Trips/m8hk-2ipk/about_data)
min_distance_city

7.91

In [41]:
threshold_moving_time = df_sf['moving_time_min'].quantile(0.99).round(2)
threshold_moving_time

36.38

In [42]:
# creating a mask for outliers based on the thresholds set above
mask_outliers = (
    (
        (df_sf['trip_distance_miles'] > threshold_distance) |   # far rides
        (df_sf['trip_distance_miles']<min_distance_city)      # short rides
    )
  &                                            # *and*
    (df_sf['trip_duration_min'] > threshold_duration)  |  (df_sf['moving_time_min'] > threshold_moving_time)       # duration > threshold
    &                                                          #  *and*
    (
        (df_sf['speed_mph'] > threshold_speed) |               # high speed
        (df_sf['speed_mph'] < 12)                              # low speed (less than 12 mph, which is the minimum moving speed for SF)
    )
)

df_sf= df_sf[~mask_outliers].copy()
print(f'Removed {mask_outliers.sum()} rows '
      f'({mask_outliers.mean()*100:.2f}% of dataset)')

Removed 4084 rows (1.17% of dataset)


In [43]:
df_sf.describe().T

,count,mean,min,25%,50%,75%,max,std
start_time_local,345394,2024-02-13 06:00:42.240476160,2024-01-15 03:33:12,2024-02-01 12:37:34,2024-02-13 18:05:28,2024-02-25 17:27:12,2024-03-07 15:28:21,NaN
end_time_local,345394,2024-02-13 06:11:47.448849920,2024-01-15 03:39:49,2024-02-01 12:50:24.249999872,2024-02-13 18:17:14,2024-02-25 17:38:04.500000,2024-03-07 15:30:24,NaN
pickup_location_latitude,345394.0,37.776679,37.709,37.766,37.782,37.788,37.81,0.018838
pickup_location_longitude,345394.0,-122.421183,-122.513,-122.431,-122.417,-122.407,-122.367,0.021475
dropoff_location_latitude,345394.0,37.775691,37.709,37.766,37.781,37.789,37.81,0.019837
dropoff_location_longitude,345394.0,-122.422426,-122.513,-122.432,-122.418,-122.408,-122.364,0.022695
sfo_pickup,345394.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
meter_fare_amount,345394.0,12.917339,0.0,8.7,11.3,15.85,109.45,6.46391
upfront_pricing,345394.0,2.599201,0.0,0.0,0.0,0.0,89.97,6.258905
promo_rate,345394.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df_sf.sort_values(by='total_fare_amount', ascending=False).head(20)

,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,dropoff_location_latitude,dropoff_location_longitude,hail_type,sfo_pickup,meter_fare_amount,upfront_pricing,...,tip,extra_amount,total_fare_amount,summary_fees,trip_distance_miles,trip_duration_min,delay_min,price_calculated_total,moving_time_min,speed_mph
232070,2024-02-21 23:06:01,2024-02-21 23:08:03,37.781,-122.471,37.780,-122.471,dispatch,0,4.80,0.0,...,0.0,330.00,334.80,330.00,0.000000,2.033333,0.004250,334.152762,2.029083,0.000000
232068,2024-02-21 23:06:01,2024-02-21 23:08:03,37.781,-122.471,37.780,-122.471,dispatch,0,4.80,0.0,...,0.0,330.00,334.80,330.00,0.000000,2.033333,0.004250,334.152762,2.029083,0.000000
249763,2024-03-04 21:17:09,2024-03-04 21:19:12,37.786,-122.420,37.787,-122.411,street,0,5.45,0.0,...,0.0,315.00,320.45,315.00,0.449905,2.050000,0.850000,321.002500,1.200000,22.495246
89866,2024-02-13 12:53:49,2024-02-13 13:30:09,37.718,-122.460,37.763,-122.457,street,0,29.50,0.0,...,0.0,5.50,281.57,252.07,4.079138,36.333333,26.816667,286.650833,9.516667,25.717857
89867,2024-02-13 12:53:49,2024-02-13 13:30:09,37.718,-122.460,37.763,-122.457,street,0,29.50,0.0,...,0.0,5.50,281.57,252.07,4.079138,36.333333,26.816667,286.650833,9.516667,25.717857
67093,2024-01-20 01:13:23,2024-01-20 01:14:11,37.778,-122.415,37.778,-122.415,street,0,4.15,0.0,...,0.0,265.85,270.00,265.85,0.000000,0.800000,0.483333,270.314167,0.316667,0.000000
208222,2024-01-31 16:15:37,2024-01-31 16:16:14,37.754,-122.392,37.754,-122.392,street,0,4.15,0.0,...,0.0,0.00,209.65,205.50,0.000000,0.616667,0.553417,210.009721,0.063250,0.000000
264268,2024-02-02 05:54:41,2024-02-02 06:02:07,37.780,-122.412,37.770,-122.414,dispatch,0,8.70,0.0,...,0.0,200.00,208.70,200.00,0.900002,7.433333,0.000000,206.750000,7.445667,7.252559
264267,2024-02-02 05:54:41,2024-02-02 06:02:07,37.780,-122.412,37.770,-122.414,dispatch,0,8.70,0.0,...,0.0,200.00,208.70,200.00,0.900002,7.433333,0.000000,206.750000,7.445667,7.252559
264284,2024-02-02 05:58:59,2024-02-02 06:05:38,37.789,-122.422,37.789,-122.422,street,0,5.45,0.0,...,0.0,200.00,205.45,200.00,0.000000,6.650000,0.000000,204.150000,6.653083,0.000000


### Final cleanup: zero‑time, zero‑distance & zero-speed, high‑fee rides  

After all statistical outlier filters were applied, a handful of records still show

* `trip_duration_min` ≈ 0 – 2  
* `trip_distance_miles` ≈ 0  
* **very high fees** (e.g., \$150 cleaning charge)

Such patterns are consistent with legitimate “incident” trips recorded by taxi fleets:

* **Cleaning/Bio‑hazard fee** when a passenger soils the vehicle  
* **Accident or breakdown** logged as a meter event but with no actual travel  
* Other *FI*–type surcharges applied at the curb

Although these rows are real billing events, they **distort price‑per‑mile and duration‑based models**, contributing high fares without corresponding travel activity.  
Therefore we flag and drop them from the analytical dataset.

## Next Steps: Handling Zero-Value Trips

We will explore two options for how to proceed:

1. **Analyze zero-value cases**  
   - Identify trips where **duration = 0**, **speed = 0** and **distance = 0**  
   - Calculate what percentage of the full dataset these represent  
   - Decide which subsets can be safely removed and which should be retained

2. **Note on speed calculation**  
   - Speed is computed only over time in motion  
   - A zero speed value may indicate:  
     - **Zero duration** (recording error)  
     - **Zero distance** (recording error)  
     - **Actual movement < 12 mph** (treated as delay time)


In [45]:
df_sf[df_sf['speed_mph'] == 0].shape[0]/df_sf.shape[0] * 100
# checking the percentage of trips with zero speed 

20.46445508607561

In [46]:
df_sf[df_sf['trip_distance_miles'] == 0].shape[0]/df_sf.shape[0] * 100  
# checking the percentage of trips with zero distance (canceled or not started trips)

5.339988534832684

In [47]:
df_sf[df_sf['trip_duration_min'] == 0].shape[0]/df_sf.shape[0] * 100 
# checking the percentage of trips with zero duration (canceled or not started trips)

0.0008685732815277625

## Zero-Value Trip Breakdown

- **Zero speed** occurs in **20%** of the dataset.  
- **Zero distance** occurs in **5.32%** of the dataset.  
- **Zero duration** occurs in **< 1%** of the dataset.  

Because zero-duration and zero-distance trips each represent less than 1% of all records, we can safely **remove** those cases. We will **retain** zero-speed trips, since:

1. **Speed** was calculated only over moving time when the taxi exceeded 12 mph.  
2. A zero speed value may simply indicate that the vehicle was moving below 12 mph (i.e. idling), not a data error.  

In [48]:
mask_delete = (df_sf['trip_duration_min'] == 0) | (df_sf['trip_distance_miles'] == 0)
print(f'Removed {mask_delete.sum()} rows '
      f'({mask_delete.mean()*100:.2f}% of dataset)')
# confirming the number of trips with zero trip duration or distance won't affect the dataset significantly
df_sf = df_sf[~mask_delete].copy()
# removing trips with zero trip duration or distance, as they are not valid trips

Removed 18444 rows (5.34% of dataset)


In [49]:
df_sf.describe().T

,count,mean,min,25%,50%,75%,max,std
start_time_local,326950,2024-02-13 05:50:48.522189824,2024-01-15 03:33:12,2024-02-01 12:56:10.249999872,2024-02-13 18:01:20.500000,2024-02-25 16:46:21.500000,2024-03-07 15:28:21,NaN
end_time_local,326950,2024-02-13 06:02:23.833635072,2024-01-15 03:39:49,2024-02-01 13:06:28.249999872,2024-02-13 18:14:21.500000,2024-02-25 16:57:46.249999872,2024-03-07 15:30:24,NaN
pickup_location_latitude,326950.0,37.77701,37.709,37.767,37.782,37.789,37.81,0.018582
pickup_location_longitude,326950.0,-122.421297,-122.513,-122.431,-122.417,-122.407,-122.367,0.021504
dropoff_location_latitude,326950.0,37.775981,37.709,37.766,37.781,37.789,37.81,0.01966
dropoff_location_longitude,326950.0,-122.422611,-122.513,-122.433,-122.418,-122.408,-122.364,0.022768
sfo_pickup,326950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
meter_fare_amount,326950.0,13.399552,0.0,8.7,11.95,16.5,104.25,6.253518
upfront_pricing,326950.0,2.744727,0.0,0.0,0.0,0.0,89.97,6.401498
promo_rate,326950.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Target Check: Fare Outliers

With duration, distance, and speed outliers removed, our remaining target column — **`total_fare_amount`** — still contains a few values that exceed reasonable fare limits.  
Next we examine these high‑fare points to decide whether they are legitimate (e.g., long freeway trips) or residual anomalies that should be handled before modelling.

In [50]:
df_sf.sort_values(by='total_fare_amount', ascending=False).head(20)
# investigating the trips with the highest total fare amount

,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,dropoff_location_latitude,dropoff_location_longitude,hail_type,sfo_pickup,meter_fare_amount,upfront_pricing,...,tip,extra_amount,total_fare_amount,summary_fees,trip_distance_miles,trip_duration_min,delay_min,price_calculated_total,moving_time_min,speed_mph
249763,2024-03-04 21:17:09,2024-03-04 21:19:12,37.786,-122.420,37.787,-122.411,street,0,5.45,0.00,...,0.00,315.00,320.45,315.00,0.449905,2.050000,0.850000,321.002500,1.200000,22.495246
89866,2024-02-13 12:53:49,2024-02-13 13:30:09,37.718,-122.460,37.763,-122.457,street,0,29.50,0.00,...,0.00,5.50,281.57,252.07,4.079138,36.333333,26.816667,286.650833,9.516667,25.717857
89867,2024-02-13 12:53:49,2024-02-13 13:30:09,37.718,-122.460,37.763,-122.457,street,0,29.50,0.00,...,0.00,5.50,281.57,252.07,4.079138,36.333333,26.816667,286.650833,9.516667,25.717857
264268,2024-02-02 05:54:41,2024-02-02 06:02:07,37.780,-122.412,37.770,-122.414,dispatch,0,8.70,0.00,...,0.00,200.00,208.70,200.00,0.900002,7.433333,0.000000,206.750000,7.445667,7.252559
264267,2024-02-02 05:54:41,2024-02-02 06:02:07,37.780,-122.412,37.770,-122.414,dispatch,0,8.70,0.00,...,0.00,200.00,208.70,200.00,0.900002,7.433333,0.000000,206.750000,7.445667,7.252559
249751,2024-03-04 21:15:02,2024-03-04 21:16:54,37.781,-122.422,37.786,-122.420,street,0,5.45,0.00,...,0.00,105.00,110.45,105.00,0.469901,1.866667,0.600000,110.840000,1.266667,22.258454
198056,2024-01-31 02:39:07,2024-01-31 03:15:37,37.790,-122.421,37.748,-122.429,street,0,104.25,0.00,...,0.00,5.50,109.75,5.50,18.000045,36.500000,0.000000,68.150000,36.511583,29.579727
198054,2024-01-31 02:39:07,2024-01-31 03:15:37,37.790,-122.421,37.748,-122.429,street,0,104.25,0.00,...,0.00,5.50,109.75,5.50,18.000045,36.500000,0.000000,68.150000,36.511583,29.579727
55115,2024-02-11 06:44:48,2024-02-11 07:00:49,37.764,-122.404,37.802,-122.437,flywheel_uber,0,17.15,89.97,...,0.00,0.00,89.97,0.00,4.026672,16.016667,16.016667,27.560833,0.000000,0.000000
55114,2024-02-11 06:44:48,2024-02-11 07:00:49,37.764,-122.404,37.802,-122.437,flywheel_uber,0,17.15,89.97,...,0.00,0.00,89.97,0.00,4.026672,16.016667,16.016667,27.560833,0.000000,0.000000


In [51]:
threshold_fees = df_sf['summary_fees'].quantile(0.99).round(2)
threshold_fees


# setting the threshold for summary fees based on the 99th percentile of summary fees distribution

6.25

## Post-Cleanup: Target Column Inspection

- After cleaning the dataset, we still have our target column **Total Fare Amount**, which now reveals some unusually large values.
- A quick inspection (sorting by Total Fare Amount) shows that these extreme outliers are driven by the **Extra Amount**  and **Summary Fees** columns values.
  - In the 99th percentile, **Summary Fees** is 6.25
  - However, **< 1%** of records have **Summary Fees&** in the hundreds or even thousands of dollars.
- Since Waymo fares do not include any “Extra Amount” or fee component, we will drop those outlier rows to prevent them from introducing noise into our analysis.

### Adding the *difference* feature  

We created a new column  **'difference'** that represnts the difference between 'total_fare_amount' (the actual price paid) and 'price_calculated_total' columns.
This value measures how far the recorded fare deviates from the tariff‑based price (including all fees and tips).
Next steps:
1.	Flag outliers: rows where difference exceeds a chosen tolerance.
2.	Inspect those rows: large positive or negative gaps often signal data‑entry errors, stuck meters, or missing fee fields.
3.	Clean: drop or correct the confirmed anomalies before modelling.

In [52]:
df_sf = df_sf[df_sf['summary_fees'] <= threshold_fees].copy()
# removing trips with summary fees above the threshold, as they will skew the analysis 

In [53]:
upfront= df_sf[df_sf['upfront_pricing']>0].copy()
# checking the trips with upfront pricing >0
upfront['difference'] =upfront['total_fare_amount']- upfront['upfront_pricing'] 
# calculating the difference between total fare amount and upfront pricing to see if there are any discrepancies

In [54]:
upfront['difference'].describe().T
# checking the distribution of the diffrence between total fare amount and upfrond price

count    58134.000000
mean         1.006686
std          1.710833
min          0.000000
25%          0.000000
50%          0.000000
75%          1.350000
max          6.250000
Name: difference, dtype: float64

In [55]:
sns.histplot(upfront['difference'], bins=30, kde=True)
plt.xlabel('Difference')
plt.ylabel('Frequency')
plt.title('Distribution of Difference between Total Fare Amount and Upfront Pricing for Upfront Paid Trips')

Text(0.5, 1.0, 'Distribution of Difference between Total Fare Amount and Upfront Pricing for Upfront Paid Trips')

In [56]:
upfront.sort_values(by='difference', ascending=False).head(20)
# checking the trips with the highest difference between total fare amount and upfront pricing

,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,dropoff_location_latitude,dropoff_location_longitude,hail_type,sfo_pickup,meter_fare_amount,upfront_pricing,...,extra_amount,total_fare_amount,summary_fees,trip_distance_miles,trip_duration_min,delay_min,price_calculated_total,moving_time_min,speed_mph,difference
59614,2024-02-11 14:43:28,2024-02-11 15:12:38,37.778,-122.465,37.743,-122.417,flywheel_app,0,28.58,28.20,...,0.0,34.45,6.25,5.248891,29.166667,15.966667,42.678333,13.200000,23.858595,6.25
10961,2024-02-07 08:03:48,2024-02-07 08:35:52,37.746,-122.415,37.790,-122.444,flywheel_app,0,27.90,28.20,...,0.0,34.45,6.25,4.589030,32.066667,20.483333,43.014167,11.583333,23.770517,6.25
90905,2024-02-13 13:33:36,2024-02-13 13:56:37,37.800,-122.407,37.741,-122.403,flywheel_app,0,27.08,28.20,...,0.0,34.45,6.25,5.268887,23.016667,13.466667,41.053333,9.550000,33.102953,6.25
308128,2024-02-25 01:50:52,2024-02-25 02:13:31,37.773,-122.412,37.738,-122.486,flywheel_app,0,28.34,28.20,...,0.0,34.45,6.25,6.458635,22.650000,7.366667,40.988333,15.283333,25.355602,6.25
277720,2024-02-03 22:58:47,2024-02-03 23:12:28,37.783,-122.432,37.747,-122.432,flywheel_app,0,15.85,15.85,...,0.0,22.10,6.25,2.809406,13.683333,6.100000,28.465000,7.583333,22.228270,6.25
69076,2024-02-12 09:12:06,2024-02-12 09:22:28,37.779,-122.440,37.787,-122.448,flywheel_app,0,11.98,12.60,...,0.0,18.85,6.25,2.119552,10.366667,4.950000,25.117500,5.416667,23.478114,6.25
219232,2024-03-02 18:49:55,2024-03-02 19:04:39,37.766,-122.426,37.784,-122.388,flywheel_app,0,15.35,15.85,...,0.0,22.10,6.25,2.649440,14.733333,7.883333,28.974167,6.850000,23.206775,6.25
146416,2024-01-27 10:18:09,2024-01-27 10:29:29,37.790,-122.430,37.790,-122.430,flywheel_app,0,13.20,12.60,...,0.0,18.85,6.25,1.739632,11.333333,6.883333,25.074167,4.450000,23.455718,6.25
329861,2024-02-26 23:34:46,2024-02-26 23:48:55,37.781,-122.412,37.775,-122.454,flywheel_app,0,15.55,15.85,...,0.0,22.10,6.25,2.449482,14.150000,8.200000,28.530000,5.950000,24.700663,6.25
308041,2024-02-25 01:39:46,2024-02-25 01:52:03,37.756,-122.419,37.731,-122.450,flywheel_app,0,16.75,15.85,...,0.0,22.10,6.25,3.189326,12.283333,4.283333,27.934167,8.000000,23.919945,6.25


### Up‑Front Fare Consistency  

* For trips with **`upfront_pricing > 0`**, the up‑front amount and `total_fare_amount` are almost identical.  
* Minor discrepancies appear only on rides with **high `delay_min`** (delay_min == trip_duration_min) most —likely extra waiting that the fixed quote didn’t fully cover.

In [57]:
df_sf['difference'] = df_sf['total_fare_amount'] - df_sf['price_calculated_total']
# calculating the difference between total fare amount and calculated price to check if there are any discrepancies between the two

In [58]:
df_sf['difference'].describe().T
# distribution of the difference between total fare amount and calculated price

count    324640.000000
mean          0.551363
std           4.218415
min         -37.721667
25%          -1.350000
50%           1.297053
75%           2.600000
max          62.409167
Name: difference, dtype: float64

In [59]:
sns.histplot(df_sf['difference'], bins=30, kde=True)
plt.xlabel('Difference')
plt.ylabel('Frequency')
plt.title('Distribution of Difference between Total Fare Amount and Calculated Price')
# plotting the distribution of the difference between total fare amount and calculated price

Text(0.5, 1.0, 'Distribution of Difference between Total Fare Amount and Calculated Price')

In [60]:
threshold_difference = df_sf['difference'].quantile(0.99).round(2)
threshold_difference
# setting the threshold for difference based on the 99th percentile of difference distribution

9.75

In [61]:
df_sf = df_sf[df_sf['difference'].abs() <= threshold_difference].copy()
# checking the distribution of the absolute amount of the difference between total fare amount and calculated price

In [62]:
df_sf.sort_values(by='total_fare_amount', ascending=False).head(20)

,start_time_local,end_time_local,pickup_location_latitude,pickup_location_longitude,dropoff_location_latitude,dropoff_location_longitude,hail_type,sfo_pickup,meter_fare_amount,upfront_pricing,...,extra_amount,total_fare_amount,summary_fees,trip_distance_miles,trip_duration_min,delay_min,price_calculated_total,moving_time_min,speed_mph,difference
153159,2024-01-27 22:19:01,2024-01-27 22:57:53,37.794,-122.396,37.793,-122.396,street,0,69.80,0.0,...,0.0,69.80,0.0,18.500046,38.866667,0.010683,63.956944,38.855983,28.567100,5.843056
71601,2024-02-12 11:51:35,2024-02-12 12:23:00,37.795,-122.402,37.807,-122.415,street,0,62.00,0.0,...,5.5,67.50,5.5,16.200057,31.416667,0.000000,62.300000,31.423017,30.932849,5.200000
71602,2024-02-12 11:51:35,2024-02-12 12:23:00,37.795,-122.402,37.807,-122.415,street,0,62.00,0.0,...,5.5,67.50,5.5,16.200040,31.416667,0.000000,62.300000,31.423017,30.932817,5.200000
340712,2024-02-28 11:47:26,2024-02-28 12:20:29,37.710,-122.484,37.710,-122.484,street,0,66.55,0.0,...,0.0,66.55,0.0,17.900071,33.050000,0.000000,62.000000,33.060933,32.485600,4.550000
205414,2024-03-02 00:19:13,2024-03-02 00:55:56,37.797,-122.405,37.771,-122.408,dispatch,0,66.55,0.0,...,0.0,66.55,0.0,17.200043,36.716667,0.000000,60.050000,36.719933,28.104696,6.500000
340711,2024-02-28 11:47:26,2024-02-28 12:20:29,37.710,-122.484,37.710,-122.484,street,0,66.55,0.0,...,0.0,66.55,0.0,17.900044,33.050000,0.000000,62.000000,33.060933,32.485552,4.550000
205416,2024-03-02 00:19:13,2024-03-02 00:55:56,37.797,-122.405,37.771,-122.408,dispatch,0,66.55,0.0,...,0.0,66.55,0.0,17.200032,36.716667,0.000000,60.050000,36.719933,28.104679,6.500000
153315,2024-02-16 16:21:12,2024-02-16 16:57:10,37.755,-122.430,37.790,-122.427,street,0,60.70,0.0,...,5.5,66.20,5.5,15.100041,35.966667,0.003967,58.402578,35.962700,25.192838,7.797422
153316,2024-02-16 16:21:12,2024-02-16 16:57:10,37.755,-122.430,37.790,-122.427,street,0,60.70,0.0,...,5.5,66.20,5.5,15.100038,35.966667,0.003967,58.402578,35.962700,25.192832,7.797422
6798,2024-02-06 11:23:31,2024-02-06 11:52:43,37.717,-122.485,37.807,-122.413,street,0,60.05,0.0,...,5.5,65.55,5.5,15.700039,29.200000,0.004733,60.353077,29.195267,32.265584,5.196923


In [63]:
df_sf.describe().T

,count,mean,min,25%,50%,75%,max,std
start_time_local,314038,2024-02-13 06:57:04.722259968,2024-01-15 03:33:12,2024-02-01 12:29:31.249999872,2024-02-13 19:05:42,2024-02-25 19:56:58.249999872,2024-03-07 15:28:21,NaN
end_time_local,314038,2024-02-13 07:08:17.916423680,2024-01-15 03:39:49,2024-02-01 12:39:47,2024-02-13 19:18:28.500000,2024-02-25 20:08:03.249999872,2024-03-07 15:30:24,NaN
pickup_location_latitude,314038.0,37.777303,37.709,37.768,37.783,37.789,37.81,0.018264
pickup_location_longitude,314038.0,-122.421037,-122.513,-122.43,-122.417,-122.408,-122.367,0.021087
dropoff_location_latitude,314038.0,37.776268,37.709,37.766,37.782,37.789,37.81,0.019405
dropoff_location_longitude,314038.0,-122.422379,-122.513,-122.432,-122.418,-122.408,-122.364,0.022418
sfo_pickup,314038.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
meter_fare_amount,314038.0,13.069957,0.0,8.7,11.95,15.85,69.8,5.966411
upfront_pricing,314038.0,2.26986,0.0,0.0,0.0,0.0,53.78,5.608017
promo_rate,314038.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
report_clean_sf = ProfileReport(df_sf, title="San Francisco Cleaned Data Report", explorative=True)
report_clean_sf.to_file("San_Francisco_Cleaned_Data_Report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:00<00:00, 99.27it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [65]:
comparison_report = report_clean_sf.compare(report_raw)
comparison_report.to_file("San_Francisco_Comparison_Report.html")

/opt/anaconda3/lib/python3.12/site-packages/ydata_profiling/compare_reports.py:191: UserWarning: The datasets being profiled have a different set of columns. Only the left side profile will be calculated.
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]